## Imports

In [ ]:
# Required imports

from __future__ import division
import numpy as np
import sys
from sys import platform as sys_pf
if sys_pf == 'Darwin':
    import matplotlib
    matplotlib.use("TkAgg")

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from matplotlib import animation
import math
import ar
from sklearn.preprocessing import normalize
import pandas as pd
from pandas import Series, DataFrame, Panel
import seaborn as sns
import scipy.signal

## Helper Functions

### Plotting functions

In [ ]:
def plot_trajectory_2D(n, is_one, x, y,cxx=None,cyy=None, step=1, scatter_plot=False,title='MUST'):
    """************************************************************** 
    n:        no_of_frames to plot
    is_one:   True:  Single object's trajectory
              False: Multiple object's trajectories
    step:     step_size
    x,y:      Coordinates to plot
    **************************************************************"""
    if is_one:
        traj_points = 1
        T = np.linspace(0,1,x.shape[0])

    else : 
        traj_points = x.shape[0]
        T = np.linspace(0,1,traj_points)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    s = step
    plt.gca().invert_yaxis()
    for index in range(traj_points): 
        for i in range(0,n-s,s):
            if is_one : 
                ax.plot(x[i:i+s+1],y[i:i+s+1],linewidth=3)
            else : 
                
                cx = x[index]
                cy = y[index]
                if (scatter_plot):
                    ax.scatter(cx[i:i+s+1],cy[i:i+s+1],color='b')
                else:
                    ax.plot(cx[i:i+s+1],cy[i:i+s+1],linewidth=3,color='b')
    if (cxx is not None):
        n=cxx.shape[1]
        for index in range(traj_points): 
            for i in range(0,n-s,s):
                if is_one : 
                    ax.plot(x[i:i+s+1],y[i:i+s+1],linewidth=3)
                else : 

                    cx = cxx[index]
                    cy = cyy[index]
                    if (scatter_plot):
                        ax.scatter(cx[i:i+s+1],cy[i:i+s+1],color='r')
                    else:
                        ax.plot(cx[i:i+s+1],cy[i:i+s+1],linewidth=3,color='r')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(title)
    plt.show()

In [ ]:
def plot_trajectory_3D(n, is_one, x, y, z, step=1):
    """************************************************************** 
    n:        no_of_frames to plot
    is_one:   True:  Single object's trajectory
              False: Multiple object's trajectories
    step:     step_size
    x,y:      Coordinates to plot
    **************************************************************"""
    
    fig = plt.figure() 
    ax = fig.add_subplot(111, projection='3d')
    plt.gca().invert_yaxis()
    

    if is_one :
        traj_points = 1
        T = np.linspace(0,1,x.shape[0])

    else : 
        traj_points = x.shape[0]
        T = np.linspace(0,1,traj_points)

    s = step
    for index in range(traj_points):
        for j in range(0, n-s, s):
            if is_one :             
                ax.plot(x[j:j+s+1],y[j:j+s+1],z[j:j+s+1],linewidth=2)
            else : 
                cx = x[index]
                cy = y[index]
                cz = z
                ax.plot(cx[j:j+s+1],cy[j:j+s+1],cz[j:j+s+1],linewidth=2,color=(0.0,0.0,T[index]))


    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    plt.show()

### Predict functionality for AR

In [ ]:
def predict(X, A, Q,order=2, predict_states=1):
    try:
        Xnext = np.zeros((X.shape[0],X.shape[1]+predict_states))
        Xnext[:, :order] = X[:, :order]


        #Assuming np.size(X, axis = 1) = order
        for p in range(np.size(X, axis = 1),np.size(X, axis = 1)+predict_states):
            for j in range(0, order):
                Xnext[:, p] +=np.dot(A[j], Xnext[:, p - j - 1])
    
        orig_X = Xnext[:,:order]
        next_X = Xnext[:,order:]
        Xnext_mean = Xnext - np.mean(Xnext, axis = 0)
        return orig_X, next_X, Xnext, Xnext_mean
    except:
        print ("Unexpected error:", sys.exc_info())
        print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno))

        

## AR model for normal cells

Below blocks train AR model. The steps to train are:

    1. Read the input. Reshape it according to the way we wish to train the model.
    We wanted to create an aggregate model of the motion of neutrophils belonging to 
    the same class i.e. Normal/Inhibited. In our case, the input was of the following
    dimensions: 34 x 61 x 2. 34 - objects, 61 - frames, 2 - center x,y coordinates. 
    To build an aggregate model of 34 objects, we reshaped it to: 61 x 68.
        
    2. Normalize / Standardize the inputs
    
    3. Project this high-dimeNsional data into low-dimensional space, using PCA.
    
    4. Invoke ar.train()
    
    5. Invoke ar.test()

### Step 1: Read inputs

This block just reads all the trajectories from input files. It also stacks the trajectories of all objects. For eg: If we have 20 objects, trajectories length: 100 frames and each object is represented by its center x,y coordinates, output will be: 20 x 100 x 2.

In [ ]:

normal_cells_files = [
        '/media/narita/Data/Neutrophils/fps_20_frames/1_07052016_Phagocytosis/final' \
                      '/tracking/organized_kalman_output_center_coords.npy',
        '/media/narita/Data/Neutrophils/fps_20_frames/5_07072016_Phagocytosis_DMSO/final' \
                      '/tracking/organized_kalman_output_center_coords.npy'


]

all_normal_cells = []
for i in range(len(normal_cells_files)):
    normal_input = np.load(normal_cells_files[i])
    all_normal_cells.insert(i,normal_input)

all_normal_cells = np.vstack(all_normal_cells) 


#### Trajectory Plots

Just plotting the stacked input to make sure trajectories look correct.

In [ ]:
is_one = False
plot_trajectory_2D(all_normal_cells.shape[1],is_one,all_normal_cells[:,:,0],all_normal_cells[:,:,1],title='Normal')


### Step 2: Normalize Inputs

In [ ]:
all_normal_cells_copy = all_normal_cells.copy()
all_normal_cells_copy = np.transpose(all_normal_cells_copy,(1,0,2))
all_normal_cells_copy = all_normal_cells_copy.reshape(all_normal_cells_copy.shape[0],all_normal_cells_copy.shape[1]*all_normal_cells_copy.shape[2])
all_normal_cells_norm,normal_norms = normalize(all_normal_cells_copy,axis=0,return_norm=True)


In [ ]:
# If needed, plot again
recon = all_normal_cells_norm*normal_norms
recon = recon.reshape(all_normal_cells_copy.shape[0],int(all_normal_cells_copy.shape[1]/2),2)
recon = np.transpose(recon,(1,0,2))
plot_trajectory_2D(recon.shape[1],False,recon[:,:,0],recon[:,:,1],title='Normal')



### Step 3: State Space reduction - PCA

In [ ]:
pca_components = 3
normal_X,normal_C,normal_S,normal_U = ar.state_space(all_normal_cells_norm.T,pca_components)


#### Variance of components

Intutively, this block plots how much information is captured by every principal 
component. This helps us to decide on the number of principal components to use.

In [ ]:
diag_S = np.diag(normal_S)
tot = sum(diag_S)
var_exp = [(i / tot) * 100 for i in sorted(diag_S, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print ((var_exp))

with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(6, 4))

    plt.bar(range(len(var_exp)), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.step(range(len(var_exp)), cum_var_exp, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()
plt.show()

### Step 4: Train AR

In [ ]:
normal_X_train = normal_X[:,:]
#X_test = normal_X[:,55:]
#print (normal_X_train.shape)
#print (X_test.shape)

In [ ]:
%matplotlib inline

#ar_order is the number of past states to be used to predict next state.
ar_order = 2
is_plot=False
normal_A, normal_Q, r = ar.train(normal_X_train, order = ar_order)
r = np.array(r)
r_mean = np.mean(r,axis=0)

# Feature Vector
feature_vector = np.array(normal_A).flatten()
print ('Feature Vector:')
print (np.array(normal_A))

# Plot AR
if (is_plot):
    fig, ax = plt.subplots(1,len(normal_A), figsize=(10,3))
    for i in range(len(normal_A)):
        if (len(normal_A) > 1):
            ax[i].imshow(normal_A[i], cmap = "Blues")
            ax[i].set_xlabel('A'+str(i))
        else:
            ax.imshow(normal_A[i], cmap = "Blues")
            ax.set_xlabel('A'+str(i))
            

### Step 5: Test AR

In [ ]:
Yhat = ar.test(X_test,normal_A, normal_Q)
err = (ar.error(Yhat,X_test))
print (np.mean(err))

## AR model for MRS inhibted cells.

Following the same 5 steps for inhibted cells.

### Step 1: Read Inputs

In [ ]:
mrs_cells_files = [
        '/media/narita/Data/Neutrophils/fps_20_frames/2_07052016_Phagocytosis_MRS2578/final'\
                  '/tracking/organized_kalman_output_center_coords.npy',
        '/media/narita/Data/Neutrophils/fps_20_frames/6_07072016_Phagocytosis_MRS2578/final' \
                      '/tracking/organized_kalman_output_center_coords.npy'


]

all_mrs_cells = []
for i in range(len(mrs_cells_files)):
    mrs_input = np.load(mrs_cells_files[i])
    all_mrs_cells.insert(i,mrs_input)

all_mrs_cells = np.vstack(all_mrs_cells) 


#### Plot trajectories

In [ ]:
plot_trajectory_2D(all_mrs_cells.shape[1],False,all_mrs_cells[:,:,0],all_mrs_cells[:,:,1])


### Step 2: Normalize Input

In [ ]:
all_mrs_cells_copy = all_mrs_cells.copy()
all_mrs_cells_copy = np.transpose(all_mrs_cells_copy,(1,0,2))
all_mrs_cells_copy = all_mrs_cells_copy.reshape(all_mrs_cells_copy.shape[0],all_mrs_cells_copy.shape[1]*all_mrs_cells_copy.shape[2])

all_mrs_cells_norm,mrs_norms = normalize(all_mrs_cells_copy,axis=0,return_norm=True)


In [ ]:
# If needed, plot again
mrs_recon = all_mrs_cells_norm*mrs_norms
mrs_recon = mrs_recon.reshape(all_mrs_cells_norm.shape[0],int(all_mrs_cells_norm.shape[1]/2),2)
mrs_recon = np.transpose(mrs_recon,(1,0,2))
plot_trajectory_2D(mrs_recon.shape[1],False,mrs_recon[:,:,0],mrs_recon[:,:,1],title='Inhibited')



### Step 3: State Space Reduction

In [ ]:
pca_components = 3
mrs_X,mrs_C,mrs_S,mrs_U = ar.state_space(all_mrs_cells_norm.T, pca_components)

#### Variance of components

In [ ]:
diag_S = np.diag(mrs_S)
tot = sum(diag_S)
var_exp = [(i / tot) * 100 for i in sorted(diag_S, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print ((var_exp))

with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(6, 4))

    plt.bar(range(len(var_exp)), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.step(range(len(var_exp)), cum_var_exp, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()
plt.show()

### Step 4: AR Train

In [ ]:
mrs_X_train = mrs_X[:,:]
#mrs_X_test = mrs_X[:,50:]

In [ ]:
is_plot=True
ar_order = 2
mrs_A, mrs_Q, r = ar.train(mrs_X_train, order = ar_order)

# Feature Vector
feature_vector = np.array(mrs_A).flatten()
#print ('Feature Vector:')
#print (np.array(mrs_A))

# Plot AR
if (is_plot):
    fig, ax = plt.subplots(1,len(mrs_A), figsize=(10,3))
    for i in range(len(mrs_A)):
        if (len(mrs_A) > 1):
            ax[i].imshow(mrs_A[i], cmap = "Blues")
            ax[i].set_xlabel('A'+str(i))
        else:
            ax.imshow(mrs_A[i], cmap = "Blues")
            ax.set_xlabel('A'+str(i))


### Step 5: AR Test

In [ ]:
Y = ar.test(mrs_X_test, A)
err = ar.error(Y,mrs_X_test)
np.mean(err)

## Histograms of components of normal, mrs

Below block plots the values taken by principal components of normal and inhibited 
cells. This shows that there is a stark difference in the distribution of values 
for normal and inhibited cells.

In [ ]:
%matplotlib inline
pcs = mrs_X.shape[0]
f, axarr = plt.subplots(1, pcs,sharey=True, sharex=True,figsize=(20,3))

for i in range(pcs):
    axarr[i].hist(normal_X[i,:],bins=7, alpha=0.5, label='normal',color='gray')
    axarr[i].hist(mrs_X[i,:],bins=7, alpha=0.6, label='inhibited',color='black')
    axarr[i].legend(loc='upper right')

plt.show()


## Noise covariance matrix Q of normal, mrs

In [ ]:
%matplotlib inline

f, axarr = plt.subplots(1, 2,figsize=(15,3))
axarr[0].imshow(normal_Q,cmap='Blues')
axarr[1].imshow(mrs_Q,cmap='Blues')

In [ ]:
def get_eigens(z,is_plot=False,is_print=False):
    cov_mat = z
    eig_vals, eig_vecs = np.linalg.eig(cov_mat)
    
    if (is_print):
        print('Eigenvectors \n%s' % eig_vecs)
        print('\nEigenvalues \n%s' % eig_vals)

        for ev in eig_vecs:
            np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
        print('Everything ok!')
        print ('')

    # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i]) for i in range(len(eig_vals))]

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    
    if (is_print):
        #Visually confirm that the list is correctly sorted by decreasing eigenvalues
        print('Eigenvalues in descending order:')
        for i in eig_pairs:
            print(i[0])

    tot = sum(eig_vals)
    var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]
    cum_var_exp = np.cumsum(var_exp)
    print ((var_exp))

    if (is_plot):
        with plt.style.context('seaborn-whitegrid'):
            plt.figure(figsize=(6, 4))

            plt.bar(range(len(var_exp)), var_exp, alpha=0.5, align='center',
                    label='individual explained variance')
            plt.step(range(len(var_exp)), cum_var_exp, where='mid',
                     label='cumulative explained variance')
            plt.ylabel('Explained variance ratio')
            plt.xlabel('Principal components')
            plt.legend(loc='best')
            plt.tight_layout()
        plt.show()




In [ ]:
print ('Normal')
get_eigens(normal_Q,is_plot=True)
print ('')

print ('Abnormal')
get_eigens(mrs_Q,is_plot=True)

## Synthesize

Below blocks synthesize sequences based on the trained models.

### Multi-Step Out-of-Sample Forecast

This forecast randomly selects a starting point and forecasts predict_states time steps ahead in the sequence from the available data used to fit the model.

#### Intialize Vars

In [ ]:
cls = 'normal'
if (cls == 'normal'):
    input_X = normal_X_train
    A = normal_A
    Q = normal_Q
    C = normal_C
    norm = normal_norms
    

else:
    
    input_X = mrs_X_train
    A = mrs_A
    Q = mrs_Q
    C = mrs_C
    norm = mrs_norms



predict_states = 100

In [ ]:
import random

selected_index = random.randrange(0, 15, 1)
print (selected_index)

X_subsample = input_X[:,selected_index:selected_index+2]
orig_X, predicted_X,orig_predicted,orig_recon = predict(X_subsample,A,Q,predict_states=predict_states)

print (orig_X.shape)
print (predicted_X.shape)
print (orig_predicted.shape)

#### Plotting to just make sure that the predictions are from the same subspace

In [ ]:
%matplotlib inline

X_in_copy = input_X.T.copy()
f, axarr = plt.subplots(1, predicted_X.shape[0],figsize=(15,3),sharex=True)
for i in range(predicted_X.shape[0]):
    X_in = X_in_copy[:,i]
    pc = list(predicted_X[i,:])
    axarr[i].hist(X_in,alpha=0.5)
    axarr[i].set_xlabel('component_'+(str(i+1)))
    axarr[i].scatter(pc,np.ones(len(pc)),color='r')

plt.show()


#### Prediciting multiple states.

In [ ]:
X_train_copy = input_X.copy()
orig_pixel_space = ar.appearance_space(X_train_copy,C)

orig_image_space = (orig_pixel_space.T * norm).T
orig_image_space = np.reshape(orig_image_space,(int(orig_image_space.shape[0]/2),2,orig_image_space.shape[1]))
orig_image_space = np.transpose(orig_image_space,(0,2,1))

predicted_copy = orig_predicted.copy()
predicted_pixel_space = ar.appearance_space(predicted_copy,C)
predicted_image_space = (predicted_pixel_space.T * norm).T
predicted_image_space = np.reshape(predicted_image_space,(int(predicted_image_space.shape[0]/2),2,predicted_image_space.shape[1]))
predicted_image_space = np.transpose(predicted_image_space,(0,2,1))

frame_num = orig_image_space.shape[1]
is_one = False
currentX = orig_image_space[:,:,0]
currentY = orig_image_space[:,:,1]


cxx = predicted_image_space[:,:,0]
cyy = predicted_image_space[:,:,1]

print (cxx.shape)

plot_trajectory_2D(frame_num,is_one,currentX,currentY,cxx,cyy,scatter_plot=True,title='Normal')
#plot_trajectory_2D(frame_num,is_one,currentX,currentY,None,None,scatter_plot=True)




In [ ]:
predicted_outputs = predicted_image_space.copy()
path = '/media/narita/Data/Neutrophils/fps_20_frames/average_motion_model' \
                      '/ar_generated_center_coords_'+cls+'_'+str(predict_states)+'.npy'

np.save(path,predicted_outputs)
print (path+' saved!')

### One-Step Out-of-Sample Forecast

A one-step forecast is a forecast of the very next time step in the sequence from the available data used to fit the model.

In [ ]:
total_frames = input_X.shape[1]
selected_index = total_frames - ar_order 
print (selected_index)

X_subsample = input_X[:,selected_index:selected_index+2]
orig_X, predicted_X,orig_predicted,orig_recon = predict(X_subsample,A,predict_states=1)

print (orig_X.shape)
print (predicted_X.shape)
print (orig_predicted.shape)

In [ ]:
X_train_copy = input_X.copy()
orig_pixel_space = ar.appearance_space(X_train_copy,C)

orig_image_space = (orig_pixel_space.T * norm).T
orig_image_space = np.reshape(orig_image_space,(int(orig_image_space.shape[0]/2),2,orig_image_space.shape[1]))
orig_image_space = np.transpose(orig_image_space,(0,2,1))

predicted_copy = orig_predicted.copy()
predicted_pixel_space = ar.appearance_space(predicted_copy,C)
predicted_image_space = (predicted_pixel_space.T * norm).T
predicted_image_space = np.reshape(predicted_image_space,(int(predicted_image_space.shape[0]/2),2,predicted_image_space.shape[1]))
predicted_image_space = np.transpose(predicted_image_space,(0,2,1))

frame_num = orig_image_space.shape[1]
is_one = False
currentX = orig_image_space[:,:,0]
currentY = orig_image_space[:,:,1]


cxx = predicted_image_space[:,:,0]
cyy = predicted_image_space[:,:,1]


plot_trajectory_2D(frame_num,is_one,currentX,currentY,cxx,cyy,scatter_plot=True)
plot_trajectory_2D(frame_num,is_one,currentX,currentY,None,None,scatter_plot=True)




## Grid Search AR

In [ ]:
Grid search determines the no. of principal components() to be used and the order of AR
model. Once these 

In [ ]:
# evaluate an AR model for a given order (o)
from sklearn.metrics import mean_squared_error

def evaluate_ar_model(X, split, ar_order):
    error = None
    try:
        # prepare training dataset
        train_size = int((X.shape[1]) * split)
        train, test = X[:,0:train_size], X[:,train_size:]
        
        # make predictions
        predictions =[]
        
        for t in range((test.shape[1])):
            _A, _Q, r = ar.train(train, order = ar_order)
            r = np.array(r)
            
            
            
            orig_X, next_X, Xnext, Xnext_mean = predict(
                                                            train[:,-ar_order:],
                                                            A=_A,
                                                            Q=_Q,
                                                            order = ar_order,
                                                            predict_states = 1
                                                        )
            
            predictions.append(next_X.flatten())
            
            z = test[:,t].reshape((test.shape[0],1))
            train = np.concatenate((train,z),axis=1)
        
        predictions = np.array(predictions).T
        
        error = mean_squared_error(predictions.flatten(), test.flatten())
    except:
        print ("Unexpected error:", sys.exc_info())
        print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno))

    return (r, _Q, error)


In [ ]:
%matplotlib inline 

# evaluate combinations of q, d values for an AR model
def evaluate_models(dataset,q_values=range(2,11), d_values=range(1,11)):
    all_res = []
    best_score, best_cfg = float("inf"), None
    split = 0.8
    cnt = 0
    for q in q_values:
        normal_X,normal_C,normal_S,normal_U = ar.state_space(dataset.T,q)
        for d in d_values:
            order = (q,d)
            try:
                res,_Q, mse = evaluate_ar_model(normal_X,split, d)
                
                #print('Current, AR%s MSE=%.10f' % (order,mse))
                if mse < best_score:
                    best_score, best_cfg = mse, order
                    print('AR%s MSE=%.10f' % (order,mse))
                    res = res.flatten()
                    plt.figure(cnt)
                    plt.imshow(_Q,cmap = "Blues")
                    plt.colorbar()
                    plt.show()
                    
            except:
                print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
                continue
         
    print('Best AR%s MSE=%.10f' % (best_cfg, best_score))
    

In [ ]:
np.random.seed(2)
evaluate_models(all_normal_cells_norm)

In [ ]:
evaluate_models(all_mrs_cells_norm)

# One - Step Prediction Error

In [ ]:
# evaluate an AR model for a given order (o)
from sklearn.metrics import mean_squared_error

def one_step_prediction(X, train_size, ar_order):
    error = None
    
    try:
        # prepare training dataset
        #train_size = int((X.shape[1]) * split)
        train, test = X[:,0:train_size], X[:,train_size:train_size+5]
        
        # make predictions
        predictions =[]
        
        for t in range((test.shape[1])):
            _A, _Q, r = ar.train(train, order = ar_order)
            r = np.array(r)
            
            
            
            orig_X, next_X, Xnext, Xnext_mean = predict(
                                                            train[:,-ar_order:],
                                                            A=_A,
                                                            Q=_Q,
                                                            order = ar_order,
                                                            predict_states = 1
                                                        )
            
            predictions.append(next_X.flatten())
            
            z = test[:,t].reshape((test.shape[0],1))
            train = np.concatenate((train,z),axis=1)
        
        predictions = np.array(predictions).T
        
        error = mean_squared_error(predictions.flatten(), test.flatten())
    except:
        print ("Unexpected error:", sys.exc_info())
        print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno))

    return (r, _Q, error)


In [ ]:
%matplotlib inline 

# evaluate combinations of q, d values for an AR model
def evaluate_one_step(dataset,d_values=range(1,5)):
    all_res = []
    best_score, best_cfg = float("inf"), None
    split = 0.8
    cnt = 0
    q = 3
    all_mse = []
    all_res = []
    for d in range(4):
        curr_res = []
        for ts in range(5,60,5):
            normal_X,normal_C,normal_S,normal_U = ar.state_space(dataset.T,q)
            order = (ts,q,d)
            try:
                res,_Q, mse = one_step_prediction(normal_X, ts, d+1)
                curr_res.append(mse)
                #print('Current, AR%s MSE=%.10f' % (order,mse))
                
            except:
                print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
                continue
        all_res.insert(d,curr_res)
    return all_res

In [ ]:
norm_res = evaluate_one_step(all_normal_cells_norm)


In [ ]:
mrs_res = evaluate_one_step(all_mrs_cells_norm)

In [ ]:
np.array(norm_res).shape

In [ ]:
fig,axarr = plt.subplots(nrows = 1, ncols=2,sharey=True, sharex=True,figsize=(12,3))

line_styles = [':','-.','--','-']

for i in range(np.array(norm_res).shape[0]):
    #print (scipy.signal.medfilt(results[i,:],7))
    axarr[0].plot(range(5,60,5),scipy.signal.medfilt(norm_res[i],7),label='AR order '+str(i+1),linestyle=line_styles[i])
    axarr[0].legend(loc='upper right')
    axarr[0].set_xlabel('#frames')
    axarr[0].set_ylabel('one-step prediction error')
    
    axarr[1].plot(range(5,60,5),scipy.signal.medfilt(mrs_res[i],7),label='AR order '+str(i+1),linestyle=line_styles[i])
    axarr[1].legend(loc='upper right')
    axarr[1].set_xlabel('#frames')

#plt.yscale('log')
plt.legend(loc=0, borderaxespad=0.5)

plt.show()